In [1]:
import numpy as np
import pandas as pd
import random
from collections import namedtuple
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [10]:
"""
    readDataset: Function that reads and randomize the dataset, returning a namedtuple -> dataset.X and dataset.Y
"""
def readDataset(filename, y_collumns):
    
    # Reading the dataset.
    data = pd.read_csv(filename, index_col=False, header=None)
    
    # Acquiring dataset data and class data.
    y = data.iloc[:,len(data.columns)-y_collumns: len(data.columns)]
    y = np.array(y)
    X = data.iloc[:,0:len(data.columns)-y_collumns]
    X = np.array(X)
    
    # Randomizing dataset.
    indices = np.random.choice(len(X), len(X), replace=False)
    X_values = X[indices]
    y_values = y[indices]
    
    # Creating an alias to dataset -> dataset.X and dataset.Y
    dataset = namedtuple('datset', 'X Y')

    return dataset(X=X_values, Y=y_values)

In [123]:
"""
    processing: Function that transform divides the dataset in train and test and transform the Y values in binary, OneHotEnconding
"""
def processing(dataset, p=0.75):
    
    # Normalizing process
    #scaler = StandardScaler()
    #scaler.fit(dataset.X)
    #x = scaler.transform(dataset.X)
    x = dataset.X
    
    # Labelizing process
    #onehot_encoder = OneHotEncoder(sparse=False)
    #y = dataset.Y.reshape(len(dataset.Y), 1)
    #y = onehot_encoder.fit_transform(y)
    y = dataset.Y
    
    # Computing the lenght of dataset
    lenght = dataset.X.shape[0]

    # Split dataset into train and test
    x_train = x[0:int(p*lenght), :]
    y_train = y[0:int(p*lenght), :]

    x_test = x[int(p*lenght):, :]
    y_test = y[int(p*lenght):, :]
        
    # Creating an alias to train and test set
    dataset = namedtuple('datset', 'X Y')
    train = dataset(X=x_train, Y=y_train)
    test = dataset(X=x_test, Y=y_test)

    return train, test

In [4]:
"""
    sigmoid: Function that applies the sigmoid function, used in the backpropagation step.
""" 
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [42]:
"""
    mlp_forward: Function that is responsible for the forward step, that applies the actual weight values on the net.
"""
def mlp_forward(x, hidden_weights, output_weights):

    f_net_h = []
    # Apllying the weights on the hidden units
    for i in range(len(hidden_weights)):
        # If is the first hidden unit
        if i == 0:
            net = np.matmul(x,hidden_weights[i][:,0:len(x)].transpose()) + hidden_weights[i][:,-1]
            f_net = sigmoid(net)
        # If is the second or more hidden unit
        else:
            net = np.matmul(f_net_h[i-1],hidden_weights[i][:,0:len(f_net_h[i-1])].transpose()) + hidden_weights[i][:,-1]
            f_net = sigmoid(net)
        
        # Store f_net of hidden layers
        f_net_h.append(f_net) 

    # Computing the net function to the output layer
    net = np.matmul(f_net_h[len(f_net_h)-1],output_weights[:,0:len(f_net_h[len(f_net_h)-1])].transpose()) + output_weights[:,-1]
        
    f_net_o = sigmoid(net)
    
    return f_net_o, f_net_h

In [165]:
"""
    mlp_backward: Function that is responsible for the backpropagation step, which corresponds to the updating of weights.
"""
def mlp_backward(dataset, j, hidden_weights, output_weights, f_net_o, f_net_h, eta, hidden_units, alpha, momentum_h, momentum_o, n_classes):

    x = dataset.X[j,:]
    y = dataset.Y[j,:]
    
    # Measuring the error
    print(f_net_o)
    print(y)
    #error = y - f_net_o
    
    y_lista = int(''.join([str(l) for l in y]))
    fneto_lista = int(''.join([str(l) for l in f_net_o]))    
    error = y_lista - fneto_lista
    
    delta_o = error*f_net_o*(1-f_net_o)
    
    # Computing the delta for the hidden units
    delta_h = []
    for i in range(len(hidden_units)-1, -1, -1):

        if(i == len(hidden_units)-1):
            w_o = output_weights[: ,0:hidden_units[i]]
            delta = (f_net_h[i]*(1-f_net_h[i]))*(np.matmul(delta_o, w_o))
        else:
            w_o = hidden_weights[i+1][:,0:hidden_units[i]]
            delta = (f_net_h[i]*(1-f_net_h[i]))*(np.matmul(delta, w_o))

        delta_h.insert(0,delta)
    
    # Computing the delta and updating weights for the output layer
    delta_o = delta_o[:, np.newaxis]
    f_net_aux = np.concatenate((f_net_h[len(hidden_units)-1],np.ones(1)))[np.newaxis, :]
    output_weights = output_weights - -2*eta*np.matmul(delta_o, f_net_aux) + momentum_o
    momentum_o = - -2*eta*np.matmul(delta_o, f_net_aux)
    
    # Updating the weights for the hidden layers
    for i in range(len(hidden_units)-1, -1, -1):
        delta = delta_h[i][:, np.newaxis]
        f_net_aux = np.concatenate((f_net_h[i],np.ones(1)))[np.newaxis, :]    

        if i == 0:
            x_aux = np.concatenate((x,np.ones(1)))[np.newaxis, :]
            hidden_weights[i] = hidden_weights[i] - -2*eta*np.matmul(delta, x_aux) + momentum_h[i]
            momentum_h[i] = - -2*eta*np.matmul(delta, x_aux)
        else:
            f_net_aux = np.concatenate((f_net_h[i-1],np.ones(1)))[np.newaxis, :]
            hidden_weights[i] = hidden_weights[i] - -2*eta*np.matmul(delta, f_net_aux) + momentum_h[i]
            momentum_h[i] = - -2*eta*np.matmul(delta, f_net_aux)

    #  Measuring the error
    error = sum(error*error)

    # Return the updated weights, the new error and the momentum parameters
    return hidden_weights, output_weights, error, momentum_h, momentum_o

In [44]:
"""
    testing: Function that is responsible to realize the tests for the classification and regression methods
             for different datasets.
"""
def testing(train, test, hidden_weights, output_weights):
    counter = 0
    for i in range(test.X.shape[0]):
        y_hat, q = mlp_forward(test.X[i,:], hidden_weights, output_weights)
        y_hat = np.argmax(y_hat)
        y = np.argmax(test.Y[i,:])
        if y == y_hat:
            counter += 1
            
    return counter/test.X.shape[0]

In [135]:
"""
    MLP: function that is responsible to initialize weights, check conditions to construct net.
"""
def MLP(dataset, hidden_units, epochs, eta, alpha, train_size):
    
    # Acquiring the train and test set
    train, test = processing(dataset, train_size)

    n_classes = len(np.unique(dataset.X))
    hidden_layers = len(hidden_units)
    
    # Initializing the weights of the hidden layers
    momentum_o = 0
    momentum_h = []
    hidden_weights = []
    for i in range(hidden_layers):
        if(i == 0):
            aux = np.zeros((hidden_units[i], dataset.X.shape[1] + 1))
        else:
            aux = np.zeros((hidden_units[i], hidden_units[i-1] + 1))
    
        hidden_weights.append(aux)
        momentum_h.append(aux)

    # Filling the hidden layers weight values with a normal distribution between -1 and 1
    for i in range(hidden_layers):
        for j in range(hidden_units[i]):
            if(i == 0):
                for k in range(dataset.X.shape[1] + 1):
                    hidden_weights[i][j][k] = random.uniform(-1, 1)
            else:
                for k in range(hidden_units[i-1]+1):
                    hidden_weights[i][j][k] = random.uniform(-1, 1)

    # Initializing and filling the weights values of output layer
    output_weights = np.zeros((n_classes, hidden_units[len(hidden_units)-1]+1))

    for i in range(n_classes):
        for j in range(hidden_units[hidden_layers-1]+1):
            output_weights[i][j] = random.uniform(-1, 1)

    epoch = 0
    for epoch in range(epochs):
        sum_errors = 0
        for i in range(train.X.shape[0]):
            # Forward
            f_net_o, f_net_h = mlp_forward(train.X[i, :], hidden_weights, output_weights)
            # Backward hidden_weights, output_weights, error = 
            hidden_weights, output_weights, error, momentum_h, momentum_o = mlp_backward(train, i, hidden_weights, output_weights, 
                                                                                        f_net_o, f_net_h, eta, hidden_units, alpha, 
                                                                                        momentum_h, momentum_o, n_classes)
            sum_errors += error
        epoch += 1
        
    # Computing the measure for the chosen method
    return testing(train, test, hidden_weights, output_weights)

____
## TESTE

In [60]:
# Dataset
dataset = readDataset("datasets/wine.csv", 1)

In [100]:
# MLP(dataset, hidden_layers, hidden_units, n_classes, epochs, eta, alpha, data_ratio):  
MLP(dataset, (1,2), 300, 1e-1, 2, 0.7)

C:\ProgramData\Miniconda3\envs\py36\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


0.8333333333333334

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset = namedtuple('datset', 'X Y')

imagens = pd.read_csv('semeion.data', sep=' ', lineterminator='\n', header=None).iloc[:,:-1]
lenght = imagens.shape[0]

imagens = dataset(X=imagens.iloc[:,:256].values, Y=imagens.iloc[:,256:].values)

In [167]:
MLP(imagens, (30,), 300, 1e-1, 2, 0.7)

[0.90996582 0.97109459]
[1 0 0 0 0 0 0 0 0 0]


ValueError: invalid literal for int() with base 10: '0.90996582017287430.9710945874749224'

In [147]:
0.join([0,1,0,0])

SyntaxError: invalid syntax (<ipython-input-147-f67527441ce2>, line 1)

In [153]:
array = ['0', '1', '_', '_', '_', '_', '_']
result = [' '.join(array)]
result

['0 1 _ _ _ _ _']

In [166]:
lista = [0,1,0,0]